In [2]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd

# Path to the CSV and where images/labels should go
csv_path = '../brain_tumor_classification/data/mapped_data/mapped_data.csv'
augmented_folder = '../brain_tumor_classification/data/augmented/'

# Create output folder if it doesn't exist
os.makedirs(augmented_folder, exist_ok=True)

# Load the existing CSV
df = pd.read_csv(csv_path)

# Augmentation settings
brightness_factors = [0.4, 0.7, 1.3, 1.6]
quality_levels = [20, 40, 60]

# List to store new rows for CSV
new_rows = []

# Helper to adjust brightness
def adjust_brightness(image, factor):
    return np.clip(image * factor, 0, 255).astype(np.uint8)

# Iterate through each image-label pair
for _, row in df.iterrows():
    image_id = row['Image_ID']
    img_path = row['Image_Path']
    label_path = row['Label_Path']

    if not os.path.exists(img_path) or not os.path.exists(label_path):
        continue  # Skip if either file is missing

    image = cv2.imread(img_path)
    img_ext = os.path.splitext(img_path)[1]

    # Quality-based augmentation
    for q in quality_levels:
        new_image_id = f"{image_id}_q{q}"
        new_image_name = f"{new_image_id}{img_ext}"
        new_label_name = f"{new_image_id}.txt"
        new_img_path = os.path.join(augmented_folder, new_image_name)
        new_lbl_path = os.path.join(augmented_folder, new_label_name)

        # Save image with different JPEG quality
        cv2.imwrite(new_img_path, image, [cv2.IMWRITE_JPEG_QUALITY, q])
        shutil.copy(label_path, new_lbl_path)

        new_rows.append({
            'Image_ID': new_image_id,
            'Image_Path': new_img_path,
            'Label_Path': new_lbl_path
        })

    # Brightness-based augmentation
    for factor in brightness_factors:
        new_image_id = f"{image_id}_b{str(factor).replace('.', '')}"
        new_image_name = f"{new_image_id}{img_ext}"
        new_label_name = f"{new_image_id}.txt"
        new_img_path = os.path.join(augmented_folder, new_image_name)
        new_lbl_path = os.path.join(augmented_folder, new_label_name)

        bright_img = adjust_brightness(image, factor)
        cv2.imwrite(new_img_path, bright_img)
        shutil.copy(label_path, new_lbl_path)

        new_rows.append({
            'Image_ID': new_image_id,
            'Image_Path': new_img_path,
            'Label_Path': new_lbl_path
        })

# Append the new rows to the DataFrame
df_augmented = pd.DataFrame(new_rows)
df_final = pd.concat([df, df_augmented], ignore_index=True)

# Save back to the same CSV
df_final.to_csv(csv_path, index=False)

print("Augmentation complete! CSV updated and new files saved to:", augmented_folder)

Augmentation complete! CSV updated and new files saved to: ../brain_tumor_classification/data/augmented/
